In [1]:
import pandas as pd
import numpy as np
import shapely
import matplotlib.pylab as plt
import geopandas as gpd
import fiona
import rtree

from geopandas import GeoDataFrame
from fiona.crs import from_epsg
from shapely.geometry import Point

%matplotlib inline

In [2]:
path = "/Users/achillessaxby/gittest_achilles/Safety_Score-Pluto_Cleaning/Data/Cleaned_DataSets/"
q_score = pd.read_csv(path + "streetscore_newyorkcity.csv")
q_score.head()

,latitude,longitude,q-score
0,40.700909,-74.013504,11.062166
1,40.701000,-74.013878,10.818611
2,40.701080,-74.012878,12.677955
3,40.701187,-74.013268,11.417325
4,40.701244,-74.012115,25.199091


In [3]:
crs = None
geometry = [Point(xy) for xy in zip(q_score.longitude, q_score.latitude)]
qscore_geo = GeoDataFrame(q_score, crs=crs, geometry=geometry)
qscore_geo.crs = fiona.crs.from_epsg(4326)

In [4]:
qscore_geo.head()

,latitude,longitude,q-score,geometry
0,40.700909,-74.013504,11.062166,POINT (-74.013504 40.700909)
1,40.701000,-74.013878,10.818611,POINT (-74.01387800000001 40.701)
2,40.701080,-74.012878,12.677955,POINT (-74.012878 40.70108)
3,40.701187,-74.013268,11.417325,POINT (-74.01326800000001 40.701187)
4,40.701244,-74.012115,25.199091,POINT (-74.01211500000001 40.701244)


In [5]:
shape_path = "/Users/achillessaxby/gittest_achilles/Safety_Score-Pluto_Cleaning/Data/qn_mappluto_16v2/QNMapPLUTO.shp"
ct_shape = GeoDataFrame.from_file(shape_path)
ct_shape.to_crs(epsg=4326, inplace=True)

In [6]:
ct_shape.head()

,APPBBL,APPDate,Address,AreaSource,AssessLand,AssessTot,BBL,BldgArea,BldgClass,BldgDepth,...,YearAlter2,YearBuilt,ZMCode,ZipCode,ZoneDist1,ZoneDist2,ZoneDist3,ZoneDist4,ZoneMap,geometry
0,0.0,None,ROCKAWAY POINT BLVD,4,2212.0,2212.0,4.163500e+09,0,V0,0.0,...,0,0,None,0,R4,None,None,None,30d,POLYGON ((-73.92849000834644 40.55783812737683...
1,0.0,None,ROCKAWAY POINT BLVD,4,2212.0,2212.0,4.163500e+09,0,V0,0.0,...,0,0,Y,0,R4,None,None,None,29b,POLYGON ((-73.92445722522712 40.55998874542297...
2,0.0,None,326 ROCKAWAY POINT BLVD,2,53787150.0,59997150.0,4.163300e+09,2000,Q0,100.0,...,0,1900,Y,11695,PARKUS,None,None,None,29d,POLYGON ((-73.88172314319606 40.56560890059463...
3,0.0,None,202-30 ROCKAWAY POINT BLVD,2,2261400.0,4831658.0,4.163500e+09,1,A8,40.0,...,0,1938,Y,11697,R4,None,None,None,29b,POLYGON ((-73.90957972084556 40.56107095138726...
4,0.0,None,ROCKAWAY POINT BLVD,4,2212.0,2212.0,4.163500e+09,0,V0,0.0,...,0,0,None,0,R4,None,None,None,29b,POLYGON ((-73.91532949798001 40.56091615483602...


In [7]:
ct_shape.columns

Index([    u'APPBBL',    u'APPDate',    u'Address', u'AreaSource',
       u'AssessLand',  u'AssessTot',        u'BBL',   u'BldgArea',
        u'BldgClass',  u'BldgDepth',  u'BldgFront',      u'Block',
         u'BoroCode',    u'Borough',   u'BsmtCode',   u'BuiltFAR',
           u'CB2010',         u'CD',     u'CT2010',    u'ComArea',
          u'CommFAR',    u'CondoNo',    u'Council',  u'EDesigNum',
        u'Easements', u'ExemptLand',  u'ExemptTot',        u'Ext',
         u'FacilFAR', u'FactryArea',   u'FireComp', u'GarageArea',
       u'HealthArea',   u'HistDist', u'IrrLotCode',    u'LandUse',
         u'Landmark',        u'Lot',    u'LotArea',   u'LotDepth',
         u'LotFront',    u'LotType',  u'LtdHeight', u'MAPPLUTO_F',
         u'NumBldgs',  u'NumFloors', u'OfficeArea',  u'OtherArea',
         u'Overlay1',   u'Overlay2',  u'OwnerName',  u'OwnerType',
       u'PLUTOMapID', u'PolicePrct',   u'ProxCode',    u'ResArea',
         u'ResidFAR', u'RetailArea', u'SHAPE_Area', u'SHAPE_Le

In [8]:
QN_Shape = ct_shape.drop([u'Borough', u'CD', u'CB2010', u'Council', u'SanitBoro', u'SanitDistr', u'SanitSub', 
                          u'Overlay1',u'Overlay2', u'SPDist1', u'SPDist2', u'SPDist3', u'LtdHeight', u'SplitZone', 
                          u'Easements', u'OwnerName', u'AreaSource', u'IrrLotCode', u'BsmtCode', u'AssessLand', 
                          u'AssessTot', u'ExemptLand', u'ExemptTot', u'YearAlter1', u'YearAlter2',u'BuiltFAR', 
                          u'ResidFAR', u'CommFAR', u'FacilFAR', u'BoroCode', u'BBL', u'CondoNo', u'Tract2010', 
                          u'ZoneMap', u'ZMCode', u'Sanborn', u'TaxMap', u'EDesigNum', u'APPBBL', u'APPDate', 
                          u'PLUTOMapID', u'Version', u'Ext', u'HistDist', u'Landmark', u'ZoneDist2', u'ZoneDist3', 
                          u'ZoneDist4'], axis = 1)

In [9]:
QN_Shape.columns

Index([   u'Address',   u'BldgArea',  u'BldgClass',  u'BldgDepth',
        u'BldgFront',      u'Block',     u'CT2010',    u'ComArea',
       u'FactryArea',   u'FireComp', u'GarageArea', u'HealthArea',
          u'LandUse',        u'Lot',    u'LotArea',   u'LotDepth',
         u'LotFront',    u'LotType', u'MAPPLUTO_F',   u'NumBldgs',
        u'NumFloors', u'OfficeArea',  u'OtherArea',  u'OwnerType',
       u'PolicePrct',   u'ProxCode',    u'ResArea', u'RetailArea',
       u'SHAPE_Area', u'SHAPE_Leng', u'SchoolDist',  u'StrgeArea',
         u'UnitsRes', u'UnitsTotal',     u'XCoord',     u'YCoord',
        u'YearBuilt',    u'ZipCode',  u'ZoneDist1',   u'geometry'],
      dtype='object')

In [10]:
QN_Shape_2 = QN_Shape.drop([u'Address', u'BldgClass', u'BldgDepth', u'ComArea', u'FactryArea',
              u'FireComp', u'GarageArea', u'HealthArea', u'Lot',u'LotDepth',u'LotFront',
              u'LotType',u'OfficeArea',  u'OtherArea',  u'OwnerType',u'PolicePrct',   
              u'ProxCode',    u'ResArea', u'RetailArea',u'SHAPE_Leng', u'SchoolDist',  
              u'StrgeArea', u'UnitsRes',u'ZoneDist1'],axis = 1)
QN_Shape_2.columns

Index([  u'BldgArea',  u'BldgFront',      u'Block',     u'CT2010',
          u'LandUse',    u'LotArea', u'MAPPLUTO_F',   u'NumBldgs',
        u'NumFloors', u'SHAPE_Area', u'UnitsTotal',     u'XCoord',
           u'YCoord',  u'YearBuilt',    u'ZipCode',   u'geometry'],
      dtype='object')

In [11]:
QN_Shape_2.head()

,BldgArea,BldgFront,Block,CT2010,LandUse,LotArea,MAPPLUTO_F,NumBldgs,NumFloors,SHAPE_Area,UnitsTotal,XCoord,YCoord,YearBuilt,ZipCode,geometry
0,0,0.0,16350,916.01,11,6200,0,0,0.0,5.813691e+03,0,0,0,0,0,POLYGON ((-73.92849000834644 40.55783812737683...
1,0,0.0,16350,916.01,11,5642,0,0,0.0,5.794274e+03,0,0,0,0,0,POLYGON ((-73.92445722522712 40.55998874542297...
2,2000,20.0,16330,916.02,09,11448960,0,53,1.0,1.304928e+07,1,1014504,143949,1900,11695,POLYGON ((-73.88172314319606 40.56560890059463...
3,1,30.0,16350,916.01,01,3589344,0,788,1.0,3.579786e+06,539,1007855,144182,1938,11697,POLYGON ((-73.90957972084556 40.56107095138726...
4,0,0.0,16350,916.01,11,5100,0,0,0.0,5.380941e+03,0,0,0,0,0,POLYGON ((-73.91532949798001 40.56091615483602...


In [12]:
QN_Shape_joined = gpd.sjoin(QN_Shape_2, qscore_geo, how="inner", op='intersects')
QN_Shape_joined.head()

,BldgArea,BldgFront,Block,CT2010,LandUse,LotArea,MAPPLUTO_F,NumBldgs,NumFloors,SHAPE_Area,UnitsTotal,XCoord,YCoord,YearBuilt,ZipCode,geometry,index_right,latitude,longitude,q-score
2,2000,20.0,16330,916.02,09,11448960,0,53,1.0,1.304928e+07,1,1014504,143949,1900,11695,POLYGON ((-73.88172314319606 40.56560890059463...,261877,40.561073,-73.901627,24.920761
2,2000,20.0,16330,916.02,09,11448960,0,53,1.0,1.304928e+07,1,1014504,143949,1900,11695,POLYGON ((-73.88172314319606 40.56560890059463...,261917,40.561726,-73.901627,23.402300
2,2000,20.0,16330,916.02,09,11448960,0,53,1.0,1.304928e+07,1,1014504,143949,1900,11695,POLYGON ((-73.88172314319606 40.56560890059463...,261906,40.561401,-73.901436,29.459480
2,2000,20.0,16330,916.02,09,11448960,0,53,1.0,1.304928e+07,1,1014504,143949,1900,11695,POLYGON ((-73.88172314319606 40.56560890059463...,261845,40.560749,-73.901436,28.340883
2,2000,20.0,16330,916.02,09,11448960,0,53,1.0,1.304928e+07,1,1014504,143949,1900,11695,POLYGON ((-73.88172314319606 40.56560890059463...,261810,40.560101,-73.901436,19.005203


In [13]:
QN_Shape_joined_2 = gpd.sjoin(qscore_geo, QN_Shape_2, how="inner", op='intersects')
QN_Shape_joined_2.head()

,latitude,longitude,q-score,geometry,index_right,BldgArea,BldgFront,Block,CT2010,LandUse,LotArea,MAPPLUTO_F,NumBldgs,NumFloors,SHAPE_Area,UnitsTotal,XCoord,YCoord,YearBuilt,ZipCode
114912,40.648342,-73.847450,19.422548,POINT (-73.84744999999999 40.648342),3741,0,0.0,14140,892,11,11600000,0,0,0.0,6.854889e+06,0,1026114,175148,0,11414
115117,40.648705,-73.847260,22.631334,POINT (-73.84725999999999 40.648705),3741,0,0.0,14140,892,11,11600000,0,0,0.0,6.854889e+06,0,1026114,175148,0,11414
115327,40.649010,-73.847427,27.093935,POINT (-73.847427 40.64901),3741,0,0.0,14140,892,11,11600000,0,0,0.0,6.854889e+06,0,1026114,175148,0,11414
115760,40.649689,-73.847443,16.700129,POINT (-73.847443 40.649689),3741,0,0.0,14140,892,11,11600000,0,0,0.0,6.854889e+06,0,1026114,175148,0,11414
116413,40.650421,-73.847839,12.577958,POINT (-73.84783900000001 40.650421),3741,0,0.0,14140,892,11,11600000,0,0,0.0,6.854889e+06,0,1026114,175148,0,11414


In [14]:
QN_Shape_joined.to_csv('QN_Geometry_Joined.csv')

In [15]:
QN_Shape_joined_2.to_csv('QN_Point_Joined.csv')